In [32]:
import numpy as np
import os
import sys
from PIL import Image
from matplotlib.pyplot import figure
import pandas
import matplotlib.pyplot as plt
import matplotlib.ticker as tick
import torch
from torch import nn
import torch.optim as optim 
from torch.utils.data import TensorDataset, DataLoader

class MLP1(nn.Module):
  '''
    Multilayer Perceptron.
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(256, 128),
      nn.ReLU(),
      nn.Linear(128, 128),
      nn.ReLU(),
      nn.Linear(128, 256),
    )


  def forward(self, x):
    '''Forward pass'''
    return self.layers(x)

class MLP2(nn.Module):
  '''
    Multilayer Perceptron.
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(1024, 512),
      nn.ReLU(),
      nn.Linear(512,256),
      nn.ReLU(),
      nn.Linear(256,128),
      nn.ReLU(),
      nn.Linear(128,24),
      nn.Softmax(dim=0),
      nn.Linear(24,256),
    )


  def forward(self, x):
    '''Forward pass'''
    return self.layers(x)

def add_noise(outputVector,noise_percent,stdev):
    '''
    Add noise to the output vector.
    '''
    mean = 0
    noise = np.random.normal(mean,stdev,outputVector.shape) * noise_percent
    return outputVector + noise

def Noisy_Testing(stdev, testRounds, inputImageVectors):
    '''
    Test the DNN with noise.
    '''
    tableObject = {}
    plotObject = {'fh': [], 'ffa': []}

    for i in range(len(stdev)):
        tableObject['std_'+ str(stdev[i]) + '_fh'] = []
        tableObject['std_'+ str(stdev[i]) + '_ffa'] = []

    for j in range(len(stdev)):   
        for k in range(testRounds) :
            corruptedVector = add_noise(inputImageVectors[k],0.1,stdev[j]) 
            testPrediction = model(torch.from_numpy(corruptedVector.astype('float32'))).detach().numpy()
            for l in range(256):
                if output[l] > 0:
                    testPrediction[l] = 1
                else:
                    testPrediction[l] = 0
            fh,ffa = calculate_performance_metrics(inputImageVectors[k],testPrediction)
            print("FH:",fh)
            print("FFA:",ffa)

def Create_Image_Set(filename, ASL):
    ImageVectors = []
    if filename!='ASL32x' and filename!='ASL64x' and ASL==False: #set1 and set2
        for i in range(10):
            path = os.path.join(os.path.dirname(os.path.abspath(sys.argv[1])),filename, str(i) +'.png')
            im = Image.open(path, 'r')
            gray = im.convert('L')
            bw = gray.point(lambda x: 0 if x<135 else 1, '1')
            ImageVectors.append(np.array(list(bw.getdata())))
        for i in range(26):
            x = i + 65
            path = os.path.join(os.path.dirname(os.path.abspath(sys.argv[1])),filename, chr(i + 65) +'.png')
            im = Image.open(path, 'r')
            gray = im.convert('L')
            bw = gray.point(lambda x: 0 if x<135 else 1, '1')
            ImageVectors.append(np.array(list(bw.getdata())))
    
    elif (filename=='ASL32x' or filename=='ASL64x') and ASL==False: #set3
        for i in range(25):
            x = i + 65
            if i!=9:
                path = os.path.join(os.path.dirname(os.path.abspath(sys.argv[1])),filename, chr(i + 65) +'.png')
                im = Image.open(path, 'r')
                gray = im.convert('L')
                bw = gray.point(lambda x: 0 if x<135 else 1, '1')
                ImageVectors.append(np.array(list(bw.getdata())))
    
    elif ASL==True: #set1mod
        for i in range(25):
            x = i + 65
            if i!=9:
                path = os.path.join(os.path.dirname(os.path.abspath(sys.argv[1])),filename, chr(i + 65) +'.png')
                print(path)
                im = Image.open(path, 'r')
                gray = im.convert('L')
                bw = gray.point(lambda x: 0 if x<135 else 1, '1')
                ImageVectors.append(np.array(list(bw.getdata())))

    return np.array(ImageVectors)

set1 = Create_Image_Set('characters1',False)
set2 = Create_Image_Set('characters2',False)
set3 = Create_Image_Set('ASL32x',False)
#set3 = Create_Image_Set('ASL64x',False)
set1mod = Create_Image_Set('characters1',True)

imageTensor = torch.Tensor(set1)
_dataSet = TensorDataset(imageTensor, imageTensor)
_dataLoader = DataLoader(_dataSet)

model = MLP1()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(10):
    losses = []

    for batch_idx, (data, targets) in enumerate(_dataLoader):

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()
    
def calculate_performance_metrics(inputVector, outputVector):
    totalBlackPixelCount = sum(x == 0 for x in inputVector)
    totalWhitePixelCount = sum(x == 1 for x in inputVector)
    wrongBlackPixelCount = 0
    rightBlackPixelCount = 0
    
    for i in range(256):
        if outputVector[i] < 0.0001:
            if  abs(outputVector[i] - inputVector[i]) < 0.0001:
                rightBlackPixelCount += 1
            else:
                wrongBlackPixelCount += 1
    fh = rightBlackPixelCount/totalBlackPixelCount
    ffa = wrongBlackPixelCount/totalWhitePixelCount
    return fh, ffa 


model.eval()
print("\n------PART 1-------\n")
print("\nChecking accuracy on Training Set\n")
for i in range(36):
    output = model(torch.from_numpy(set1[i].astype('float32'))).detach().numpy()
    for j in range(256):
        if output[j] > 0:
            output[j] = 1
        else:
            output[j] = 0
    fh, ffa = calculate_performance_metrics(set1[i], output)
    print("FH:", fh, "FFA:", ffa)

print("\nChecking accuracy on Test Set\n")
for i in range(36):
    output = model(torch.from_numpy(set2[i].astype('float32'))).detach().numpy()
    for j in range(256):
        if output[j] > 0:
            output[j] = 1
        else:
            output[j] = 0
    fh, ffa = calculate_performance_metrics(set2[i], output)
    print("FH:", fh, "FFA:", ffa)
print("\nNOISY\n")
stdev = [0,0.001, 0.002, 0.003, 0.005, 0.01, 0.02, 0.03, 0.05,0.1]
Noisy_Testing(stdev, 10, set1)
print("\n-------PART 2--------\n")
imageTensor = torch.Tensor(set3)
resultTensor = torch.Tensor(set1mod)
print("Image tensor dimension:",imageTensor.size())
print("Result tensor dimension:", resultTensor.size())
_dataSet = TensorDataset(imageTensor, resultTensor)
_dataLoader = DataLoader(_dataSet)

model = MLP2()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(20):
    losses = []

    for batch_idx, (data, targets) in enumerate(_dataLoader):

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

for i in range(24):
    output = model(torch.from_numpy(set3[i].astype('float32'))).detach().numpy()
    for j in range(256):
        if output[j] > 0:
            output[j] = 1
        else:
            output[j] = 0
    fh, ffa = calculate_performance_metrics(set1mod[i], output)
    print("FH:", fh, "FFA:", ffa)

C:\Users\vjsol\Desktop\UF\CAP6615\CAP6615Group\Assignment2\characters1\A.png
C:\Users\vjsol\Desktop\UF\CAP6615\CAP6615Group\Assignment2\characters1\B.png
C:\Users\vjsol\Desktop\UF\CAP6615\CAP6615Group\Assignment2\characters1\C.png
C:\Users\vjsol\Desktop\UF\CAP6615\CAP6615Group\Assignment2\characters1\D.png
C:\Users\vjsol\Desktop\UF\CAP6615\CAP6615Group\Assignment2\characters1\E.png
C:\Users\vjsol\Desktop\UF\CAP6615\CAP6615Group\Assignment2\characters1\F.png
C:\Users\vjsol\Desktop\UF\CAP6615\CAP6615Group\Assignment2\characters1\G.png
C:\Users\vjsol\Desktop\UF\CAP6615\CAP6615Group\Assignment2\characters1\H.png
C:\Users\vjsol\Desktop\UF\CAP6615\CAP6615Group\Assignment2\characters1\I.png
C:\Users\vjsol\Desktop\UF\CAP6615\CAP6615Group\Assignment2\characters1\K.png
C:\Users\vjsol\Desktop\UF\CAP6615\CAP6615Group\Assignment2\characters1\L.png
C:\Users\vjsol\Desktop\UF\CAP6615\CAP6615Group\Assignment2\characters1\M.png
C:\Users\vjsol\Desktop\UF\CAP6615\CAP6615Group\Assignment2\characters1\N.png

FH: 0.3624161073825503 FFA: 0.4672897196261682
FH: 0.3968253968253968 FFA: 0.43283582089552236
FH: 0.3885350318471338 FFA: 0.43434343434343436
FH: 0.42105263157894735 FFA: 0.36363636363636365
FH: 0.38990825688073394 FFA: 0.5
FH: 0.418848167539267 FFA: 0.36923076923076925
FH: 0.4126984126984127 FFA: 0.3880597014925373
FH: 0.391304347826087 FFA: 0.4444444444444444
FH: 0.42727272727272725 FFA: 0.3904109589041096
FH: 0.39664804469273746 FFA: 0.42857142857142855
FH: 0.40625 FFA: 0.40625
FH: 0.4147465437788018 FFA: 0.358974358974359
FH: 0.40703517587939697 FFA: 0.40350877192982454
FH: 0.4121212121212121 FFA: 0.3956043956043956
FH: 0.39325842696629215 FFA: 0.4358974358974359
FH: 0.40522875816993464 FFA: 0.4077669902912621
FH: 0.40703517587939697 FFA: 0.40350877192982454
FH: 0.43783783783783786 FFA: 0.323943661971831
FH: 0.4411764705882353 FFA: 0.36666666666666664
FH: 0.4 FFA: 0.4225352112676056
FH: 0.37748344370860926 FFA: 0.44761904761904764
FH: 0.3333333333333333 FFA: 0.569620253164557
FH: 